Langchain 프롬프트 템플릿
Langchain 프롬프트 템플릿 하는 이유 : 

```
LLM 프롬프트를 동적으로 구성하고 재사용할 수 있도록 해주는 도구

        단일 입력 : 하나의 변수로 구성된 프롬프트 템플릿
        다중 입력 : 둘 이상의 변수를 사용하는 템플릿
        ChatPromptTemplate 역할 기반 프롬프트 : 시스템/사용자 역할별 프롬프트 구성 .from_message()
        PartialPromptTemplate 활용 : 프롬프트 일부를 미리 고정하고 부분포맷티을 사용(ex. 시스템 메시지는 고정)
        프롬프트 출력 및 체인 실행 : LCEL
        프롬프트 작성 팁 : 주의사항 및 모범사례
```


In [ ]:
%pip install langchain-openai langchain-community python-dotenv openai

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# 단일 프롬프트 사용
from langchain.prompts import PromptTemplate
# 템플릿 문자열 정의
template_str = (
    "당신은 최고 수준의 마케팅 카피라이터 입니다. \n"
    "아래 제품의 매력적인 홍보문구를 100자 내외로 작성해 주세요.\n\n "
    "제품명 : {product_name}"
)
# PromptTemplate 객체 생성
product_prompt = PromptTemplate.from_template(template_str)
# 프롬프트 템플릿에 값 채우기
formatted_prompt = product_prompt.format(product_name="이클립스")
# 프롬프트 출력    
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터 입니다. 
아래 제품의 매력적인 홍보문구를 100자 내외로 작성해 주세요.

 제품명 : 이클립스


## LLM과 연결

In [9]:
# 프롬프트 | LLM 연결 -> invoke
from langchain_openai import ChatOpenAI
# OpenAI LLM 연결
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# Runnable 객체 생성 LCEL
chain = product_prompt | llm
result = chain.invoke({"product_name": "이클립스"})
# 결과 출력
print(result.content)

"이클립스, 당신의 일상에 빛나는 변화를! 매일의 순간을 특별하게 만들어주는 혁신적인 경험을 만나보세요. 지금 바로 이클립스와 함께하세요!"


In [10]:
# 다중 입력
# 다중 입력 템프릿 문자열 정의
multi_template_str = (
    "아래는 뉴스 기사 제목과 키워드 입니다.\n"
    "이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요.\n\n"
    "제목 : {title}\n"
    "키워드 : {keyword}"
)

# 프롬프트 템플릿 작성
summary_prompt = PromptTemplate(template=multi_template_str, input_variables=["title", "keyword"])
# 프롬프트 템플릿에 값 채우기 (포맷팅을 통해 프롬프트 값 확인)
sample_title = "인공지능 기술의 발전과 미래"
sample_keyword = "머신러닝, 딥러닝, LLM, 랭체인, 산업 혁신"

formatted_summary_prompt = summary_prompt.format(
    title = sample_title,
    keyword = sample_keyword
)
print(formatted_summary_prompt)

아래는 뉴스 기사 제목과 키워드 입니다.
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요.

제목 : 인공지능 기술의 발전과 미래
키워드 : 머신러닝, 딥러닝, LLM, 랭체인, 산업 혁신


In [13]:
# 문자열 출력 파서 Runnable 객체  - invoke
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser() # 문자열 출력 파서 생성

# LCEL 출력
summary_chain = summary_prompt | llm | parser
# invoke 메서드로 프롬프트 실행 
summary_result = summary_chain.invoke({
    "title": sample_title,
    "keyword": sample_keyword
})
# 결과 출력
summary_result

'인공지능 기술의 발전은 머신러닝과 딥러닝의 혁신을 통해 가속화되고 있으며, 특히 대규모 언어 모델(LLM)과 랭체인 기술이 주목받고 있습니다. 이러한 기술들은 다양한 산업 분야에서 혁신을 이끌어내고 있으며, 기업들은 이를 활용하여 효율성을 높이고 새로운 비즈니스 모델을 창출하고 있습니다. 앞으로 인공지능은 더욱 발전하여 우리의 삶과 산업 구조에 큰 변화를 가져올 것으로 기대됩니다.'

In [ ]:
# ChainPromptTemplate과 역할 기반 프롬프트
# 시스템/사용자/어시스턴트 역할(role)
# 시스템 메시지: 머델의 동작을 지시
# 사용자 메시지: 실제 사용자의 입력
# 어시스턴트 메시지: 이전 모델이 응답한 내용이 있다면 대화맥락유지를 위해 활용

In [ ]:
from langchain.prompts import ChatPromptTemplate
# 역할 기반 프롬프트 템플릿 생성    
system_message = ("당신은 python 분야의 뛰어난 전문가이자 조언자 입니다."
                  "사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요.")
user_message = "{question}"
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", user_message)
])  

# 템플릿을 이용해서 문장을 완성해 주세요 -> LLM 쓰지않음 (chat_prompt는 Runnable 객체)
sample_question = "파이썬의 클래스에 대해서 설명해주세요"
formatted_chat_prompt = chat_prompt.format_messages(question=sample_question)
# 프롬프트 출력
print(formatted_chat_prompt)



# 파이프라인을 이용해서 LLM호출 및 출력

chat_chain = chat_prompt | llm | parser
# invoke 메서드로 프롬프트 실행
chat_result = chat_chain.invoke({
    "question": "파이썬에서 리스트를 정렬하는 방법은?"
})
# # 결과 출력
# print(chat_result)


[SystemMessage(content='당신은 python 분야의 뛰어난 전문가이자 조언자 입니다.사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='파이썬의 클래스에 대해서 설명해주세요', additional_kwargs={}, response_metadata={})]
